# **installation de la bibliotheque NEO4j**

In [ ]:
! pip install neo4j
! pip install sqlite3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.8 MB/s eta 0:00:00


# Migration de données du modèle relationnel vers la base postgres

In [ ]:
from neo4j import GraphDatabase
import sqlite3

# Connexion à la base SQLite
sqlite_conn = sqlite3.connect("infractions.db")
sqlite_cursor = sqlite_conn.cursor()

# Connexion à Neo4j
neo4j_uri = "bolt://localhost:7687"  # Remplace par l'URI de ton instance Neo4j
neo4j_user = "neo4j"  # Nom d'utilisateur de Neo4j
neo4j_password = "password"  # Mot de passe de Neo4j

driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Fonction pour insérer un nœud
def create_node(tx, label, properties):
    query = f"CREATE (n:{label} {{ {', '.join([f'{key}: ${key}' for key in properties])} }})"
    tx.run(query, **properties)

# Fonction pour insérer une relation
def create_relationship(tx, start_label, start_key, end_label, end_key, rel_type):
    query = f"""
    MATCH (a:{start_label} {{ {start_key[0]}: ${start_key[1]} }}), (b:{end_label} {{ {end_key[0]}: ${end_key[1]} }})
    CREATE (a)-[:{rel_type}]->(b)
    """
    tx.run(query, **{start_key[1]: start_key[2], end_key[1]: end_key[2]})

# Migration des données
with driver.session() as session:

    # Table infractions -> Noeuds (Infraction)
    sqlite_cursor.execute("SELECT code_infr, lib_infr FROM infractions")
    for code_infr, lib_infr in sqlite_cursor.fetchall():
        session.write_transaction(create_node, "Infraction", {"code_infr": code_infr, "lib_infr": lib_infr})

    # Table départements -> Noeuds (Département)
    sqlite_cursor.execute("SELECT code_dep, nom_dep FROM départements")
    for code_dep, nom_dep in sqlite_cursor.fetchall():
        session.write_transaction(create_node, "Departement", {"code_dep": code_dep, "nom_dep": nom_dep})

    # Table force_odre -> Noeuds (ForceOrdre)
    sqlite_cursor.execute("SELECT force_odre FROM force_odre")
    for force_odre, in sqlite_cursor.fetchall():
        session.write_transaction(create_node, "ForceOrdre", {"force_odre": force_odre})

    # Table brigade -> Noeuds (Brigade) avec relations vers Département et ForceOrdre
    sqlite_cursor.execute("SELECT id_brigade, lib_brigade, lib_perimetre, code_dep, force_odre FROM brigade")
    for id_brigade, lib_brigade, lib_perimetre, code_dep, force_odre in sqlite_cursor.fetchall():
        session.write_transaction(create_node, "Brigade", {
            "id_brigade": id_brigade,
            "lib_brigade": lib_brigade,
            "lib_perimetre": lib_perimetre,
            "code_dep": code_dep,
            "force_odre": force_odre
        })

    # Table fait -> Relation entre Brigade et Infraction
    sqlite_cursor.execute("SELECT code_infr, id_brigade, annee, nb_infraction FROM fait")
    for code_infr, id_brigade, annee, nb_infraction in sqlite_cursor.fetchall():
        session.write_transaction(create_relationship,
                                 "Brigade", ("id_brigade", "id_brigade", id_brigade),
                                 "Infraction", ("code_infr", "code_infr", code_infr),
                                 "A_COMMIS")

# Fermeture des connexions
sqlite_conn.close()
driver.close()

print("Migration terminée avec succès.")
